# The Revenge of Rocchio's Angels

We will be #1 this time

Local Script Dependencies

In [1]:
!python -c "import platform; print(platform.platform()); import sys; print(sys.version)"

Windows-10-10.0.26200-SP0
3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]


In [2]:
from pathlib import Path
from engine import SearchEngine

C:\Users\idoli\PycharmProjects\FinalProject\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
se = SearchEngine()

In [4]:
se.set_searcher()

In [5]:
results = se.get_top_k("international organized crime", k=50)

In [6]:
print(results[0][0])
print(results[0][1][:300])
print(results[0][2])

FBIS3-24145
<TEXT>
Language: <F P=105> Russian </F>
Article Type:BFN

<F P=106> [Feature comprising address to Russia's citizens from the </F>
Russian Ministry of Internal Affairs and "Materials from"
"Report by the Ministry of Internal Affairs to Russia's
Citizens" under the "ROSSIYSKAYA GAZETA Exclusive" rubr
3.115000009536743


In [7]:
def build_trec_run(
    queries_path: Path,
    output_path: Path,
    engine,
    k: int = 1000,
    run_tag: str = "run1",
    num_queries: int = -1,
):
    """
    Reads queries file: each line 'topic_id\\tquery text'
    Writes TREC run file with 6 columns.
    """
    lines_out = []
    with queries_path.open("r", encoding="utf-8") as f:
        for line_no, line in enumerate(f, start=1):
            line = line.strip()
            if not line:
                continue
            try:
                topic, query = line.split("\t", 1)
            except ValueError:
                raise ValueError(f"Bad format at line {line_no}: expected 'topic\\tquery'. Got: {line!r}")

            topic = topic.strip()
            query = query.strip()

            hits = [(res[0],res[-1]) for res in engine.get_top_k(query, k)]
            hits.sort(key=lambda x: x[1], reverse=True)
            for rank, (docid, score) in enumerate(hits, start=1):
                # topic  Q0  docid  rank  score  run_tag
                lines_out.append(f"{topic} Q0 {docid} {rank} {score:.2f} {run_tag}")
            if line_no == num_queries:
                break

    output_path.parent.mkdir(parents=True, exist_ok=True)
    output_path.write_text("\n".join(lines_out) + "\n", encoding="utf-8")


In [8]:
build_trec_run(Path("Data/queriesROBUST.txt"), Path("run_1.res"), se, k=2, num_queries=50)